In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

In [1]:
# !pip install pyspark
import pyspark

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local") \
    .appName("Spark DataFrames") \  # możliwość podglądu zadania po nazwie w przeglądarce ('localhost:4040')
    .getOrCreate()

23/08/12 08:56:14 WARN Utils: Your hostname, md-ASUS resolves to a loopback address: 127.0.1.1; using 192.168.30.211 instead (on interface eth0)
23/08/12 08:56:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/12 08:56:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/12 08:56:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
spark.sparkContext.getConf().getAll()  # sprawdzenie konfiguracji (domyślne wartości parametrów)

[('spark.master', 'local'),
 ('spark.app.startTime', '1691823376337'),
 ('spark.app.id', 'local-1691823378086'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.port', '44759'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDi

In [4]:
!spark-submit --version

23/08/12 08:56:36 WARN Utils: Your hostname, md-ASUS resolves to a loopback address: 127.0.1.1; using 192.168.30.211 instead (on interface eth0)
23/08/12 08:56:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.4.1
      /_/
                        
Using Scala version 2.12.17, OpenJDK 64-Bit Server VM, 1.8.0_382
Branch HEAD
Compiled by user centos on 2023-06-19T23:01:01Z
Revision 6b1ff22dde1ead51cbf370be6e48a802daae58b6
Url https://github.com/apache/spark
Type --help for more information.


# Creating DF (DataFrame)

In [10]:
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row

df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [11]:
df.show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  4|5.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+



In [6]:
df = spark.createDataFrame([
    (1, 2., 'string1', date(2000, 1, 1), datetime(2000, 1, 1, 12, 0)),
    (2, 3., 'string2', date(2000, 2, 1), datetime(2000, 1, 2, 12, 0)),
    (3, 4., 'string3', date(2000, 3, 1), datetime(2000, 1, 3, 12, 0))
], schema='a long, b double, c string, d date, e timestamp')
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [12]:
df.show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  4|5.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+



In [14]:
# konwersja 'pandas' DataFrame() do formatu 'Spark' DataFrame

pandas_df = pd.DataFrame({
    'a': [1, 2, 3],
    'b': [2., 3., 4.],
    'c': ['string1', 'string2', 'string3'],
    'd': [date(2000, 1, 1), date(2000, 2, 1), date(2000, 3, 1)],
    'e': [datetime(2000, 1, 1, 12, 0), datetime(2000, 1, 2, 12, 0), datetime(2000, 1, 3, 12, 0)]
})

df = spark.createDataFrame(pandas_df)
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [15]:
df.show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  3|4.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+



In [16]:
#  https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#data-types

from pyspark.sql.types import (StructType, 
                               StructField, 
                               StringType, 
                               IntegerType, 
                               BooleanType, 
                               DoubleType, 
                               TimestampType
                              )

data = [(1.5, "James", 35, True, datetime(2000, 1, 1, 12, 0))]

schema = StructType([ \
    StructField("id", DoubleType()), 
    StructField("name", StringType()), 
    StructField("age", IntegerType()),
    StructField("is_client", BooleanType()),
    StructField("date", TimestampType())
  ])
 
df = spark.createDataFrame(data=data, schema=schema)
df

DataFrame[id: double, name: string, age: int, is_client: boolean, date: timestamp]

In [19]:
# wyświetlanie schematu tabeli ('schema')

df.printSchema()
df.show(truncate=False)  # brak ucinania wartości w długich kolumnach (domyślnie 'True')

root
 |-- id: double (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- is_client: boolean (nullable = true)
 |-- date: timestamp (nullable = true)

+---+-----+---+---------+-------------------+
|id |name |age|is_client|date               |
+---+-----+---+---------+-------------------+
|1.5|James|35 |true     |2000-01-01 12:00:00|
+---+-----+---+---------+-------------------+



In [24]:
# Saving / reading data

#df.write.csv('dummy_2')  # zapis danych do oddzielnego folderu (w formacie Spark DataFrame)

df = spark.read.csv('dummy_2')
df

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string]

# DataFrame's details

In [26]:
hac_df = spark.read.csv('HappinessAlcoholConsumption.csv', header=True, inferSchema=True)
hac_df.show(5)

+-----------+--------------+----------+--------------+---+-------------+--------------+----------------+--------------+
|    Country|        Region|Hemisphere|HappinessScore|HDI|GDP_PerCapita|Beer_PerCapita|Spirit_PerCapita|Wine_PerCapita|
+-----------+--------------+----------+--------------+---+-------------+--------------+----------------+--------------+
|    Denmark|Western Europe|     north|         7.526|928|       53.579|           224|              81|           278|
|Switzerland|Western Europe|     north|         7.509|943|       79.866|           185|             100|           280|
|    Iceland|Western Europe|     north|         7.501|933|        60.53|           233|              61|            78|
|     Norway|Western Europe|     north|         7.498|951|        70.89|           169|              71|           129|
|    Finland|Western Europe|     north|         7.413|918|       43.433|           263|             133|            97|
+-----------+--------------+----------+-

**Displaying**

In [42]:
hac_df.head()

Row(Country='Denmark', Region='Western Europe', Hemisphere='north', HappinessScore=7.526, HDI=928, GDP_PerCapita=53.579, Beer_PerCapita=224, Spirit_PerCapita=81, Wine_PerCapita=278)

In [41]:
hac_df.show(5)

+-----------+--------------+----------+--------------+---+-------------+--------------+----------------+--------------+
|    Country|        Region|Hemisphere|HappinessScore|HDI|GDP_PerCapita|Beer_PerCapita|Spirit_PerCapita|Wine_PerCapita|
+-----------+--------------+----------+--------------+---+-------------+--------------+----------------+--------------+
|    Denmark|Western Europe|     north|         7.526|928|       53.579|           224|              81|           278|
|Switzerland|Western Europe|     north|         7.509|943|       79.866|           185|             100|           280|
|    Iceland|Western Europe|     north|         7.501|933|        60.53|           233|              61|            78|
|     Norway|Western Europe|     north|         7.498|951|        70.89|           169|              71|           129|
|    Finland|Western Europe|     north|         7.413|918|       43.433|           263|             133|            97|
+-----------+--------------+----------+-

In [44]:
hac_df.show(3, vertical=True)

-RECORD 0--------------------------
 Country          | Denmark        
 Region           | Western Europe 
 Hemisphere       | north          
 HappinessScore   | 7.526          
 HDI              | 928            
 GDP_PerCapita    | 53.579         
 Beer_PerCapita   | 224            
 Spirit_PerCapita | 81             
 Wine_PerCapita   | 278            
-RECORD 1--------------------------
 Country          | Switzerland    
 Region           | Western Europe 
 Hemisphere       | north          
 HappinessScore   | 7.509          
 HDI              | 943            
 GDP_PerCapita    | 79.866         
 Beer_PerCapita   | 185            
 Spirit_PerCapita | 100            
 Wine_PerCapita   | 280            
-RECORD 2--------------------------
 Country          | Iceland        
 Region           | Western Europe 
 Hemisphere       | north          
 HappinessScore   | 7.501          
 HDI              | 933            
 GDP_PerCapita    | 60.53          
 Beer_PerCapita   | 233     

In [34]:
hac_df.take(3)  # zwraca listę, umożliwia dalsze indeksowanie

[Row(Country='Denmark', Region='Western Europe', Hemisphere='north', HappinessScore=7.526, HDI=928, GDP_PerCapita=53.579, Beer_PerCapita=224, Spirit_PerCapita=81, Wine_PerCapita=278),
 Row(Country='Switzerland', Region='Western Europe', Hemisphere='north', HappinessScore=7.509, HDI=943, GDP_PerCapita=79.866, Beer_PerCapita=185, Spirit_PerCapita=100, Wine_PerCapita=280),
 Row(Country='Iceland', Region='Western Europe', Hemisphere='north', HappinessScore=7.501, HDI=933, GDP_PerCapita=60.53, Beer_PerCapita=233, Spirit_PerCapita=61, Wine_PerCapita=78)]

In [43]:
df.collect()  # wszystkie wyniki zwracane do RAM (nie należy stosować dla dużych danych!)

[Row(_c0='1.5', _c1='James', _c2='35', _c3='true', _c4='2000-01-01T12:00:00.000+01:00')]

In [45]:
hac_df.columns  # format 'pandas'

['Country',
 'Region',
 'Hemisphere',
 'HappinessScore',
 'HDI',
 'GDP_PerCapita',
 'Beer_PerCapita',
 'Spirit_PerCapita',
 'Wine_PerCapita']

In [46]:
hac_df.schema  # format 'Spark' (schema)

StructType([StructField('Country', StringType(), True), StructField('Region', StringType(), True), StructField('Hemisphere', StringType(), True), StructField('HappinessScore', DoubleType(), True), StructField('HDI', IntegerType(), True), StructField('GDP_PerCapita', DoubleType(), True), StructField('Beer_PerCapita', IntegerType(), True), StructField('Spirit_PerCapita', IntegerType(), True), StructField('Wine_PerCapita', IntegerType(), True)])

In [47]:
hac_df.printSchema()

root
 |-- Country: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Hemisphere: string (nullable = true)
 |-- HappinessScore: double (nullable = true)
 |-- HDI: integer (nullable = true)
 |-- GDP_PerCapita: double (nullable = true)
 |-- Beer_PerCapita: integer (nullable = true)
 |-- Spirit_PerCapita: integer (nullable = true)
 |-- Wine_PerCapita: integer (nullable = true)



In [48]:
hac_df.dtypes  # as a list of tupples

[('Country', 'string'),
 ('Region', 'string'),
 ('Hemisphere', 'string'),
 ('HappinessScore', 'double'),
 ('HDI', 'int'),
 ('GDP_PerCapita', 'double'),
 ('Beer_PerCapita', 'int'),
 ('Spirit_PerCapita', 'int'),
 ('Wine_PerCapita', 'int')]

In [49]:
hac_df.count()  # pandas len()

122

In [50]:
hac_df.corr('GDP_PerCapita', 'HappinessScore')

-0.451425211091511

In [51]:
hac_df.describe().show(5)

23/08/12 10:30:08 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+--------+--------------------+----------+------------------+------------------+-----------------+------------------+-----------------+-----------------+
|summary| Country|              Region|Hemisphere|    HappinessScore|               HDI|    GDP_PerCapita|    Beer_PerCapita| Spirit_PerCapita|   Wine_PerCapita|
+-------+--------+--------------------+----------+------------------+------------------+-----------------+------------------+-----------------+-----------------+
|  count|     122|                 122|       122|               122|               122|              122|               122|              122|              122|
|   mean|    null|                null|      null| 5.524827868852458| 740.8770491803278|91.48283606557379| 137.5655737704918|96.59836065573771|66.59836065573771|
| stddev|    null|                null|      null|1.1487360549077443|149.84612371102796|209.5773133103142|104.81735296565127| 81.4350280833944|88.12150325620482|
|    min| Albania|Australia 

In [52]:
hac_df.summary().show()

+-------+--------+--------------------+----------+------------------+------------------+-----------------+------------------+-----------------+-----------------+
|summary| Country|              Region|Hemisphere|    HappinessScore|               HDI|    GDP_PerCapita|    Beer_PerCapita| Spirit_PerCapita|   Wine_PerCapita|
+-------+--------+--------------------+----------+------------------+------------------+-----------------+------------------+-----------------+-----------------+
|  count|     122|                 122|       122|               122|               122|              122|               122|              122|              122|
|   mean|    null|                null|      null| 5.524827868852458| 740.8770491803278|91.48283606557379| 137.5655737704918|96.59836065573771|66.59836065573771|
| stddev|    null|                null|      null|1.1487360549077443|149.84612371102796|209.5773133103142|104.81735296565127| 81.4350280833944|88.12150325620482|
|    min| Albania|Australia 

In [54]:
pandas_df = hac_df.toPandas()  # konwersja na 'pandas' DataFrame
pandas_df

,Country,Region,Hemisphere,HappinessScore,HDI,GDP_PerCapita,Beer_PerCapita,Spirit_PerCapita,Wine_PerCapita
0,Denmark,Western Europe,north,7.526,928,53.579,224,81,278
1,Switzerland,Western Europe,north,7.509,943,79.866,185,100,280
2,Iceland,Western Europe,north,7.501,933,60.530,233,61,78
3,Norway,Western Europe,north,7.498,951,70.890,169,71,129
4,Finland,Western Europe,north,7.413,918,43.433,263,133,97
...,...,...,...,...,...,...,...,...,...
117,Tanzania,Sub-Saharan Africa,south,3.666,533,878.000,36,6,1
118,Liberia,Sub-Saharan Africa,north,3.622,432,455.000,19,152,2
119,Benin,Sub-Saharan Africa,north,3.484,512,789.000,34,4,13
120,Togo,Sub-Saharan Africa,north,3.303,500,577.000,36,2,19


# Dropping data and filling missing values

In [60]:
import pyspark.sql.functions as f

for column in hac_df.columns:
    
    nan_count = hac_df.where(f.col(column).isNull()).count()
    print(f'{column}: {nan_count}')

Country: 0
Region: 0
Hemisphere: 0
HappinessScore: 0
HDI: 0
GDP_PerCapita: 0
Beer_PerCapita: 0
Spirit_PerCapita: 0
Wine_PerCapita: 0


In [61]:
df = spark.createDataFrame([
    Row(a=1, b=float('nan'), c='string'),
    Row(a=1, b=2.0, c='string'),
    Row(a=1, b=2.0, c=None)
])

df

DataFrame[a: bigint, b: double, c: string]

In [62]:
df.distinct().show()  # distinct rows (wyrzucenie duplikatów --> tu: bez znaczenia)

+---+---+------+
|  a|  b|     c|
+---+---+------+
|  1|2.0|string|
|  1|2.0|  null|
|  1|NaN|string|
+---+---+------+



In [63]:
df.drop_duplicates(['a', 'b']).show()

+---+---+------+
|  a|  b|     c|
+---+---+------+
|  1|2.0|string|
|  1|NaN|string|
+---+---+------+



In [64]:
df.dropna().show()

+---+---+------+
|  a|  b|     c|
+---+---+------+
|  1|2.0|string|
+---+---+------+



In [65]:
df.dropna(how='any', subset=['a', 'b']).show()  # how='all' (... oraz inne opcje)

+---+---+------+
|  a|  b|     c|
+---+---+------+
|  1|2.0|string|
|  1|2.0|  null|
+---+---+------+



In [66]:
df.fillna(7.0, subset='b').show()

+---+---+------+
|  a|  b|     c|
+---+---+------+
|  1|7.0|string|
|  1|2.0|string|
|  1|2.0|  null|
+---+---+------+



In [67]:
df.fillna({"b": 8.0, "c": "X"}).show()

+---+---+------+
|  a|  b|     c|
+---+---+------+
|  1|8.0|string|
|  1|2.0|string|
|  1|2.0|     X|
+---+---+------+



## Zadanie 1: 

Wczytaj zbiór danych "salaries.csv":

- sprawdź wielkość tabeli oraz typy dla poszczególnych kolumn. 
- Sprawdź, czy są braki danych w każdej z kolumn.
- Sprawdź, czy nie ma zduplikowanych wierszy.

źródło: https://www.kaggle.com/datasets/thedevastator/know-your-worth-tech-salaries-in-2016?resource=download

In [90]:
salaries_df = spark.read.csv('salaries.csv', header=True, inferSchema=True)
salaries_df.toPandas().head()

,index,salary_id,employer_name,location_name,location_state,location_country,location_latitude,location_longitude,job_title,job_title_category,job_title_rank,total_experience_years,employer_experience_years,annual_base_pay,signing_bonus,annual_bonus,stock_value_bonus,comments,submitted_at
0,0,1,opower,"san francisco, ca",CA,US,37.77,-122.41,systems engineer,Engineering,None,13.0,2.0,125000.0,5000.0,0.0,5000 shares,Don't work here.,3/21/16 12:58
1,1,3,walmart,"bentonville, ar",AR,US,36.36,-94.20,senior developer,Software,Senior,15.0,8.0,65000.0,NaN,5000.0,"3,000",None,3/21/16 12:58
2,2,4,vertical knowledge,"cleveland, oh",OH,US,41.47,-81.67,software engineer,Software,None,4.0,1.0,86000.0,5000.0,6000.0,0,None,3/21/16 12:59
3,3,6,netapp,waltham,None,None,NaN,NaN,mts,Other,None,4.0,0.0,105000.0,5000.0,8500.0,0,None,3/21/16 13:00
4,4,12,apple,cupertino,None,None,NaN,NaN,software engineer,Software,None,4.0,3.0,110000.0,5000.0,7000.0,150000,None,3/21/16 13:02


In [70]:
salaries_df.count()  # wielkość tabeli (liczba wierszy)

1658

In [81]:
# salaries_df.dtypes
salaries_df.printSchema()

root
 |-- index: string (nullable = true)
 |-- salary_id: string (nullable = true)
 |-- employer_name: string (nullable = true)
 |-- location_name: string (nullable = true)
 |-- location_state: string (nullable = true)
 |-- location_country: string (nullable = true)
 |-- location_latitude: double (nullable = true)
 |-- location_longitude: double (nullable = true)
 |-- job_title: string (nullable = true)
 |-- job_title_category: string (nullable = true)
 |-- job_title_rank: string (nullable = true)
 |-- total_experience_years: double (nullable = true)
 |-- employer_experience_years: double (nullable = true)
 |-- annual_base_pay: double (nullable = true)
 |-- signing_bonus: double (nullable = true)
 |-- annual_bonus: double (nullable = true)
 |-- stock_value_bonus: string (nullable = true)
 |-- comments: string (nullable = true)
 |-- submitted_at: string (nullable = true)



In [69]:
import pyspark.sql.functions as f

for column in salaries_df.columns:
    
    nan_count = salaries_df.where(f.col(column).isNull()).count()
    print(f'{column}: {nan_count}')  # niektóre z kolumna zawierają braki danych

index: 0
salary_id: 0
employer_name: 6
location_name: 2
location_state: 1100
location_country: 866
location_latitude: 866
location_longitude: 866
job_title: 3
job_title_category: 3
job_title_rank: 1233
total_experience_years: 50
employer_experience_years: 50
annual_base_pay: 7
signing_bonus: 326
annual_bonus: 322
stock_value_bonus: 405
comments: 1366
submitted_at: 6


In [92]:
salaries_df.distinct().count() == salaries_df.count()  # brak duplikatów

True

# Adding, removing and modifying DataFrame columns

In [94]:
hac_df.Country  # 'lazy evaluations'!!

Column<'Country'>

In [95]:
hac_df.columns

['Country',
 'Region',
 'Hemisphere',
 'HappinessScore',
 'HDI',
 'GDP_PerCapita',
 'Beer_PerCapita',
 'Spirit_PerCapita',
 'Wine_PerCapita']

**Selecting specified columns**

In [96]:
hac_df.select(hac_df.Country).show(5)

+-----------+
|    Country|
+-----------+
|    Denmark|
|Switzerland|
|    Iceland|
|     Norway|
|    Finland|
+-----------+
only showing top 5 rows



In [97]:
hac_df.select("Country").show(5)

+-----------+
|    Country|
+-----------+
|    Denmark|
|Switzerland|
|    Iceland|
|     Norway|
|    Finland|
+-----------+
only showing top 5 rows



In [98]:
import pyspark.sql.functions as f

hac_df.select(f.col('Country')).show(5)  # funkcja 'col' pozwala interpretować napis jako nazwę kolumny (czasem jest to ważne)

+-----------+
|    Country|
+-----------+
|    Denmark|
|Switzerland|
|    Iceland|
|     Norway|
|    Finland|
+-----------+
only showing top 5 rows



In [99]:
hac_df.select(["Country", "Region"]).show(5)

+-----------+--------------+
|    Country|        Region|
+-----------+--------------+
|    Denmark|Western Europe|
|Switzerland|Western Europe|
|    Iceland|Western Europe|
|     Norway|Western Europe|
|    Finland|Western Europe|
+-----------+--------------+
only showing top 5 rows



In [100]:
hac_df.drop("Country", "Region").show()

+----------+--------------+---+-------------+--------------+----------------+--------------+
|Hemisphere|HappinessScore|HDI|GDP_PerCapita|Beer_PerCapita|Spirit_PerCapita|Wine_PerCapita|
+----------+--------------+---+-------------+--------------+----------------+--------------+
|     north|         7.526|928|       53.579|           224|              81|           278|
|     north|         7.509|943|       79.866|           185|             100|           280|
|     north|         7.501|933|        60.53|           233|              61|            78|
|     north|         7.498|951|        70.89|           169|              71|           129|
|     north|         7.413|918|       43.433|           263|             133|            97|
|     north|         7.404|922|       42.349|           240|             122|           100|
|     north|         7.339|928|       45.638|           251|              88|           190|
|     south|         7.334|915|       40.332|           203|          

**Creating new column based on existing one**

In [101]:
from pyspark.sql.functions import upper  # wielkie litery ('uppercase')

hac_df.withColumn('upper_Country', upper(hac_df.Country)).show(5)

+-----------+--------------+----------+--------------+---+-------------+--------------+----------------+--------------+-------------+
|    Country|        Region|Hemisphere|HappinessScore|HDI|GDP_PerCapita|Beer_PerCapita|Spirit_PerCapita|Wine_PerCapita|upper_Country|
+-----------+--------------+----------+--------------+---+-------------+--------------+----------------+--------------+-------------+
|    Denmark|Western Europe|     north|         7.526|928|       53.579|           224|              81|           278|      DENMARK|
|Switzerland|Western Europe|     north|         7.509|943|       79.866|           185|             100|           280|  SWITZERLAND|
|    Iceland|Western Europe|     north|         7.501|933|        60.53|           233|              61|            78|      ICELAND|
|     Norway|Western Europe|     north|         7.498|951|        70.89|           169|              71|           129|       NORWAY|
|    Finland|Western Europe|     north|         7.413|918|    

In [102]:
hac_df.withColumn("HDI+1", hac_df.HDI + 1).select(['HDI', 'HDI+1']).show(5)

+---+-----+
|HDI|HDI+1|
+---+-----+
|928|  929|
|943|  944|
|933|  934|
|951|  952|
|918|  919|
+---+-----+
only showing top 5 rows



In [103]:
hac_df.withColumnRenamed("Country", "Place").show(5)

+-----------+--------------+----------+--------------+---+-------------+--------------+----------------+--------------+
|      Place|        Region|Hemisphere|HappinessScore|HDI|GDP_PerCapita|Beer_PerCapita|Spirit_PerCapita|Wine_PerCapita|
+-----------+--------------+----------+--------------+---+-------------+--------------+----------------+--------------+
|    Denmark|Western Europe|     north|         7.526|928|       53.579|           224|              81|           278|
|Switzerland|Western Europe|     north|         7.509|943|       79.866|           185|             100|           280|
|    Iceland|Western Europe|     north|         7.501|933|        60.53|           233|              61|            78|
|     Norway|Western Europe|     north|         7.498|951|        70.89|           169|              71|           129|
|    Finland|Western Europe|     north|         7.413|918|       43.433|           263|             133|            97|
+-----------+--------------+----------+-

**Replacing value**

In [104]:
hac_df.replace('Denmark', 'Other', subset=['Country']).show(5)

+-----------+--------------+----------+--------------+---+-------------+--------------+----------------+--------------+
|    Country|        Region|Hemisphere|HappinessScore|HDI|GDP_PerCapita|Beer_PerCapita|Spirit_PerCapita|Wine_PerCapita|
+-----------+--------------+----------+--------------+---+-------------+--------------+----------------+--------------+
|      Other|Western Europe|     north|         7.526|928|       53.579|           224|              81|           278|
|Switzerland|Western Europe|     north|         7.509|943|       79.866|           185|             100|           280|
|    Iceland|Western Europe|     north|         7.501|933|        60.53|           233|              61|            78|
|     Norway|Western Europe|     north|         7.498|951|        70.89|           169|              71|           129|
|    Finland|Western Europe|     north|         7.413|918|       43.433|           263|             133|            97|
+-----------+--------------+----------+-

**Aliasing columns**

In [105]:
hac_df.select(hac_df.Region.name('Place'), hac_df.Country).show(5)

+--------------+-----------+
|         Place|    Country|
+--------------+-----------+
|Western Europe|    Denmark|
|Western Europe|Switzerland|
|Western Europe|    Iceland|
|Western Europe|     Norway|
|Western Europe|    Finland|
+--------------+-----------+
only showing top 5 rows



In [106]:
hac_df.select(hac_df.Region.alias('Place'), hac_df.Country).show(5)

+--------------+-----------+
|         Place|    Country|
+--------------+-----------+
|Western Europe|    Denmark|
|Western Europe|Switzerland|
|Western Europe|    Iceland|
|Western Europe|     Norway|
|Western Europe|    Finland|
+--------------+-----------+
only showing top 5 rows



## Zadanie 2: 

W zbiorze danych `salary`: 
- zmien nazwy kolumn: `total_experience_years` -> `tot_exp_years`, `employer_experience_years` -> `emp_exp_years`
- stwórz kolumnę `annual_total`, która wynosi `annual_base_pay + annual_bonus`
- usuń kolumnę `salary_id`
- zamień wartość "apple" w kolumnie `employer_name` na "orange"

In [121]:
import pyspark.sql.functions as f

salaries_df.show(5)  # "brzydkie" wyświetlania, więc potem korzystamy z 'toPandas().head()'

+-----+---------+------------------+-----------------+--------------+----------------+-----------------+------------------+-----------------+------------------+--------------+----------------------+-------------------------+---------------+-------------+------------+-----------------+----------------+-------------+
|index|salary_id|     employer_name|    location_name|location_state|location_country|location_latitude|location_longitude|        job_title|job_title_category|job_title_rank|total_experience_years|employer_experience_years|annual_base_pay|signing_bonus|annual_bonus|stock_value_bonus|        comments| submitted_at|
+-----+---------+------------------+-----------------+--------------+----------------+-----------------+------------------+-----------------+------------------+--------------+----------------------+-------------------------+---------------+-------------+------------+-----------------+----------------+-------------+
|    0|        1|            opower|san francisco

In [124]:
# salaries_df.withColumnRenamed("total_experience_years", "tot_exp_years") \
#            .withColumnRenamed("employer_experience_years", "emp_exp_years") \
#            .show(5)

salaries_df.withColumnsRenamed({"total_experience_years": "tot_exp_years", "employer_experience_years": "emp_exp_years"}) \
    .toPandas().head()

,index,salary_id,employer_name,location_name,location_state,location_country,location_latitude,location_longitude,job_title,job_title_category,job_title_rank,tot_exp_years,emp_exp_years,annual_base_pay,signing_bonus,annual_bonus,stock_value_bonus,comments,submitted_at
0,0,1,opower,"san francisco, ca",CA,US,37.77,-122.41,systems engineer,Engineering,None,13.0,2.0,125000.0,5000.0,0.0,5000 shares,Don't work here.,3/21/16 12:58
1,1,3,walmart,"bentonville, ar",AR,US,36.36,-94.20,senior developer,Software,Senior,15.0,8.0,65000.0,NaN,5000.0,"3,000",None,3/21/16 12:58
2,2,4,vertical knowledge,"cleveland, oh",OH,US,41.47,-81.67,software engineer,Software,None,4.0,1.0,86000.0,5000.0,6000.0,0,None,3/21/16 12:59
3,3,6,netapp,waltham,None,None,NaN,NaN,mts,Other,None,4.0,0.0,105000.0,5000.0,8500.0,0,None,3/21/16 13:00
4,4,12,apple,cupertino,None,None,NaN,NaN,software engineer,Software,None,4.0,3.0,110000.0,5000.0,7000.0,150000,None,3/21/16 13:02


In [126]:
# salaries_df.withColumn("annual_total", sacolies_df['annual_base_pay'] + salaries_df['annual_bonus']).toPandas().head()
salaries_df.withColumn("annual_total", f.col('annual_base_pay') + f.col('annual_bonus')).toPandas().head()

,index,salary_id,employer_name,location_name,location_state,location_country,location_latitude,location_longitude,job_title,job_title_category,job_title_rank,total_experience_years,employer_experience_years,annual_base_pay,signing_bonus,annual_bonus,stock_value_bonus,comments,submitted_at,annual_total
0,0,1,opower,"san francisco, ca",CA,US,37.77,-122.41,systems engineer,Engineering,None,13.0,2.0,125000.0,5000.0,0.0,5000 shares,Don't work here.,3/21/16 12:58,125000.0
1,1,3,walmart,"bentonville, ar",AR,US,36.36,-94.20,senior developer,Software,Senior,15.0,8.0,65000.0,NaN,5000.0,"3,000",None,3/21/16 12:58,70000.0
2,2,4,vertical knowledge,"cleveland, oh",OH,US,41.47,-81.67,software engineer,Software,None,4.0,1.0,86000.0,5000.0,6000.0,0,None,3/21/16 12:59,92000.0
3,3,6,netapp,waltham,None,None,NaN,NaN,mts,Other,None,4.0,0.0,105000.0,5000.0,8500.0,0,None,3/21/16 13:00,113500.0
4,4,12,apple,cupertino,None,None,NaN,NaN,software engineer,Software,None,4.0,3.0,110000.0,5000.0,7000.0,150000,None,3/21/16 13:02,117000.0


In [119]:
salaries_df.drop("salary_id").toPandas().head()

,index,employer_name,location_name,location_state,location_country,location_latitude,location_longitude,job_title,job_title_category,job_title_rank,total_experience_years,employer_experience_years,annual_base_pay,signing_bonus,annual_bonus,stock_value_bonus,comments,submitted_at
0,0,opower,"san francisco, ca",CA,US,37.77,-122.41,systems engineer,Engineering,None,13.0,2.0,125000.0,5000.0,0.0,5000 shares,Don't work here.,3/21/16 12:58
1,1,walmart,"bentonville, ar",AR,US,36.36,-94.20,senior developer,Software,Senior,15.0,8.0,65000.0,NaN,5000.0,"3,000",None,3/21/16 12:58
2,2,vertical knowledge,"cleveland, oh",OH,US,41.47,-81.67,software engineer,Software,None,4.0,1.0,86000.0,5000.0,6000.0,0,None,3/21/16 12:59
3,3,netapp,waltham,None,None,NaN,NaN,mts,Other,None,4.0,0.0,105000.0,5000.0,8500.0,0,None,3/21/16 13:00
4,4,apple,cupertino,None,None,NaN,NaN,software engineer,Software,None,4.0,3.0,110000.0,5000.0,7000.0,150000,None,3/21/16 13:02


In [120]:
salaries_df.replace('apple', 'orange', subset=['employer_name']).toPandas().head()

,index,salary_id,employer_name,location_name,location_state,location_country,location_latitude,location_longitude,job_title,job_title_category,job_title_rank,total_experience_years,employer_experience_years,annual_base_pay,signing_bonus,annual_bonus,stock_value_bonus,comments,submitted_at
0,0,1,opower,"san francisco, ca",CA,US,37.77,-122.41,systems engineer,Engineering,None,13.0,2.0,125000.0,5000.0,0.0,5000 shares,Don't work here.,3/21/16 12:58
1,1,3,walmart,"bentonville, ar",AR,US,36.36,-94.20,senior developer,Software,Senior,15.0,8.0,65000.0,NaN,5000.0,"3,000",None,3/21/16 12:58
2,2,4,vertical knowledge,"cleveland, oh",OH,US,41.47,-81.67,software engineer,Software,None,4.0,1.0,86000.0,5000.0,6000.0,0,None,3/21/16 12:59
3,3,6,netapp,waltham,None,None,NaN,NaN,mts,Other,None,4.0,0.0,105000.0,5000.0,8500.0,0,None,3/21/16 13:00
4,4,12,orange,cupertino,None,None,NaN,NaN,software engineer,Software,None,4.0,3.0,110000.0,5000.0,7000.0,150000,None,3/21/16 13:02


In [128]:
salaries_df2 = salaries_df \
                .withColumnRenamed("total_experience_years", "tot_exp_years") \
                .withColumnRenamed("employer_experience_years", "emp_exp_years") \
                .withColumn("annual_total", f.col("annual_base_pay") + f.col("annual_bonus")) \
                .drop("salary_id") \
                .replace("apple", "orange", subset=['employer_name'])

salaries_df2.explain(extended=True)  # plan wykonania transformacji ('lazy evaluations')

== Parsed Logical Plan ==
Project [index#4578, CASE WHEN (employer_name#4580 = apple) THEN cast(orange as string) ELSE employer_name#4580 END AS employer_name#6120, location_name#4581, location_state#4582, location_country#4583, location_latitude#4584, location_longitude#4585, job_title#4586, job_title_category#4587, job_title_rank#4588, tot_exp_years#6021, emp_exp_years#6041, annual_base_pay#4591, signing_bonus#4592, annual_bonus#4593, stock_value_bonus#4594, comments#4595, submitted_at#4596, annual_total#6061]
+- Project [index#4578, employer_name#4580, location_name#4581, location_state#4582, location_country#4583, location_latitude#4584, location_longitude#4585, job_title#4586, job_title_category#4587, job_title_rank#4588, tot_exp_years#6021, emp_exp_years#6041, annual_base_pay#4591, signing_bonus#4592, annual_bonus#4593, stock_value_bonus#4594, comments#4595, submitted_at#4596, annual_total#6061]
   +- Project [index#4578, salary_id#4579, employer_name#4580, location_name#4581, lo

In [129]:
salaries_df2.show(5)  # wykonanie planu działania, zdefiniowanego powyżej!

+-----+------------------+-----------------+--------------+----------------+-----------------+------------------+-----------------+------------------+--------------+-------------+-------------+---------------+-------------+------------+-----------------+----------------+-------------+------------+
|index|     employer_name|    location_name|location_state|location_country|location_latitude|location_longitude|        job_title|job_title_category|job_title_rank|tot_exp_years|emp_exp_years|annual_base_pay|signing_bonus|annual_bonus|stock_value_bonus|        comments| submitted_at|annual_total|
+-----+------------------+-----------------+--------------+----------------+-----------------+------------------+-----------------+------------------+--------------+-------------+-------------+---------------+-------------+------------+-----------------+----------------+-------------+------------+
|    0|            opower|san francisco, ca|            CA|              US|            37.77|         

# Filtering data

In [130]:
hac_df.where(hac_df.Country.isNotNull()).show(5)  # 'where' is an alias for 'filter'!

+-----------+--------------+----------+--------------+---+-------------+--------------+----------------+--------------+
|    Country|        Region|Hemisphere|HappinessScore|HDI|GDP_PerCapita|Beer_PerCapita|Spirit_PerCapita|Wine_PerCapita|
+-----------+--------------+----------+--------------+---+-------------+--------------+----------------+--------------+
|    Denmark|Western Europe|     north|         7.526|928|       53.579|           224|              81|           278|
|Switzerland|Western Europe|     north|         7.509|943|       79.866|           185|             100|           280|
|    Iceland|Western Europe|     north|         7.501|933|        60.53|           233|              61|            78|
|     Norway|Western Europe|     north|         7.498|951|        70.89|           169|              71|           129|
|    Finland|Western Europe|     north|         7.413|918|       43.433|           263|             133|            97|
+-----------+--------------+----------+-

In [131]:
hac_df.filter(hac_df.GDP_PerCapita < 40).show(5)

+----------+--------------------+----------+--------------+---+-------------+--------------+----------------+--------------+
|   Country|              Region|Hemisphere|HappinessScore|HDI|GDP_PerCapita|Beer_PerCapita|Spirit_PerCapita|Wine_PerCapita|
+----------+--------------------+----------+--------------+---+-------------+--------------+----------------+--------------+
|    Israel|Middle East and N...|     north|         7.267|902|       37.181|            63|              69|             9|
|Costa Rica|Latin America and...|     north|         7.087|791|       11.733|           149|              87|            11|
|    Brazil|Latin America and...|      both|         6.952|758|        8.639|           245|             145|            16|
|    Mexico|Latin America and...|     north|         6.778|772|        8.444|           238|              68|             5|
|     Chile|Latin America and...|     south|         6.705|842|       13.961|           130|             124|           172|


In [137]:
hac_df.where(hac_df.Country.contains("b") | hac_df.Country.contains("B")).select('Country').show(5)

+--------------------+
|             Country|
+--------------------+
|              Brazil|
|             Belgium|
|          Luxembourg|
|      Czech Republic|
|United Arab Emirates|
+--------------------+
only showing top 5 rows



In [138]:
hac_df.where(hac_df.Country.like('%b%') | hac_df.Country.like("%B%")).select('Country').show(5)  # składnia SQL

+--------------------+
|             Country|
+--------------------+
|              Brazil|
|             Belgium|
|          Luxembourg|
|      Czech Republic|
|United Arab Emirates|
+--------------------+
only showing top 5 rows



In [144]:
 # instrukcja warunkowa podczas tworzenia nowych kolumn (if-else)
hac_df.withColumn("happy", f.when(f.col("HappinessScore") > 7, "yes").otherwise("no")).show(20)

+-------------+--------------------+----------+--------------+---+-------------+--------------+----------------+--------------+-----+
|      Country|              Region|Hemisphere|HappinessScore|HDI|GDP_PerCapita|Beer_PerCapita|Spirit_PerCapita|Wine_PerCapita|happy|
+-------------+--------------------+----------+--------------+---+-------------+--------------+----------------+--------------+-----+
|      Denmark|      Western Europe|     north|         7.526|928|       53.579|           224|              81|           278|  yes|
|  Switzerland|      Western Europe|     north|         7.509|943|       79.866|           185|             100|           280|  yes|
|      Iceland|      Western Europe|     north|         7.501|933|        60.53|           233|              61|            78|  yes|
|       Norway|      Western Europe|     north|         7.498|951|        70.89|           169|              71|           129|  yes|
|      Finland|      Western Europe|     north|         7.413|

## Zadanie 3:

In [148]:
salaries_df.show(5)

+-----+---------+------------------+-----------------+--------------+----------------+-----------------+------------------+-----------------+------------------+--------------+----------------------+-------------------------+---------------+-------------+------------+-----------------+----------------+-------------+
|index|salary_id|     employer_name|    location_name|location_state|location_country|location_latitude|location_longitude|        job_title|job_title_category|job_title_rank|total_experience_years|employer_experience_years|annual_base_pay|signing_bonus|annual_bonus|stock_value_bonus|        comments| submitted_at|
+-----+---------+------------------+-----------------+--------------+----------------+-----------------+------------------+-----------------+------------------+--------------+----------------------+-------------------------+---------------+-------------+------------+-----------------+----------------+-------------+
|    0|        1|            opower|san francisco

Wybierz tylko te rekordy ze zbioru `salaries`, które dotycza osób `job_title_category` =  `Software` oraz pracują na stanowisku `Senior`

In [158]:
salaries_df.where((salaries_df['job_title_category'] == 'Software') & (salaries_df['job_title_rank'] == 'Senior')).toPandas().head()
# salaries_df.filter((salaries_df.job_title_category == 'Software') & (salaries_df.job_title_rank == 'Senior')).toPandas().head()

,index,salary_id,employer_name,location_name,location_state,location_country,location_latitude,location_longitude,job_title,job_title_category,job_title_rank,total_experience_years,employer_experience_years,annual_base_pay,signing_bonus,annual_bonus,stock_value_bonus,comments,submitted_at
0,1,3,walmart,"bentonville, ar",AR,US,36.36,-94.2,senior developer,Software,Senior,15.0,8.0,65000.0,NaN,5000.0,"3,000",None,3/21/16 12:58
1,7,17,knewton,new york city,NY,US,38.00,-97.0,senior software engineer,Software,Senior,8.0,2.0,135000.0,0.0,0.0,None,None,3/21/16 13:03
2,10,24,pensco trust company,"denver, co",CO,US,39.82,-105.1,senior software engineer,Software,Senior,10.0,1.5,120000.0,0.0,6000.0,0,None,3/21/16 13:04
3,26,59,quick left,boulder co,CO,US,38.00,-97.0,senior developer,Software,Senior,6.0,1.0,120000.0,0.0,0.0,500,None,3/21/16 13:08
4,65,151,b2b,nj,None,None,NaN,NaN,senior developer,Software,Senior,10.0,3.0,140000.0,0.0,10000.0,0,None,3/21/16 13:17


## Zadanie 4:

Stwórz nową kolumnę, która bedzie informacją o tym, czy w danym wierszu występuje `job_title_rank`

In [163]:
salaries_df.withColumn("has_job_title_rank", f.when(f.col("job_title_rank").isNotNull(), "true").otherwise("false")) \
           .select("job_title_rank", "has_job_title_rank").show(5)

+--------------+------------------+
|job_title_rank|has_job_title_rank|
+--------------+------------------+
|          null|             false|
|        Senior|              true|
|          null|             false|
|          null|             false|
|          null|             false|
+--------------+------------------+
only showing top 5 rows



# Grouping Data

In [164]:
df = spark.createDataFrame([
    ['red', 'banana', 1, 10], ['blue', 'banana', 2, 20], ['red', 'carrot', 3, 30],
    ['blue', 'grape', 4, 40], ['red', 'carrot', 5, 50], ['black', 'carrot', 6, 60],
    ['red', 'banana', 7, 70], ['red', 'grape', 8, 80]],
    schema=['color', 'fruit', 'v1', 'v2'])

df.show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|  red|banana|  1| 10|
| blue|banana|  2| 20|
|  red|carrot|  3| 30|
| blue| grape|  4| 40|
|  red|carrot|  5| 50|
|black|carrot|  6| 60|
|  red|banana|  7| 70|
|  red| grape|  8| 80|
+-----+------+---+---+



In [166]:
df.groupby('color').avg().show()  # 'groupby' jest aliasem dla 'groupBy'

+-----+-------+-------+
|color|avg(v1)|avg(v2)|
+-----+-------+-------+
|  red|    4.8|   48.0|
|black|    6.0|   60.0|
| blue|    3.0|   30.0|
+-----+-------+-------+



In [167]:
df.groupby('color').max('v1').show()

+-----+-------+
|color|max(v1)|
+-----+-------+
|  red|      8|
|black|      6|
| blue|      4|
+-----+-------+



In [168]:
df.groupby('color', 'fruit').mean('v2', 'v1').show()

+-----+------+-------+-------+
|color| fruit|avg(v2)|avg(v1)|
+-----+------+-------+-------+
| blue| grape|   40.0|    4.0|
|black|carrot|   60.0|    6.0|
|  red|banana|   40.0|    4.0|
|  red| grape|   80.0|    8.0|
|  red|carrot|   40.0|    4.0|
| blue|banana|   20.0|    2.0|
+-----+------+-------+-------+



In [169]:
df.groupby('color').pivot('fruit').sum('v2').show()  # 'null' dla wartości niewystępujących w DataFrame

+-----+------+------+-----+
|color|banana|carrot|grape|
+-----+------+------+-----+
|  red|    80|    80|   80|
|black|  null|    60| null|
| blue|    20|  null|   40|
+-----+------+------+-----+



In [170]:
df.groupBy(["color","fruit"]).agg(f.sum("v1").alias("v1_sum"), f.avg("v2").alias("v2_avg"), f.count("v1")).show()

+-----+------+------+------+---------+
|color| fruit|v1_sum|v2_avg|count(v1)|
+-----+------+------+------+---------+
| blue| grape|     4|  40.0|        1|
|black|carrot|     6|  60.0|        1|
|  red|banana|     8|  40.0|        2|
|  red| grape|     8|  80.0|        1|
|  red|carrot|     8|  40.0|        2|
| blue|banana|     2|  20.0|        1|
+-----+------+------+------+---------+



In [175]:
df.groupBy(["color","fruit"]).agg((f.sum("v1") - f.avg("v2"))).show()

+-----+------+-------------------+
|color| fruit|(sum(v1) - avg(v2))|
+-----+------+-------------------+
| blue| grape|              -36.0|
|black|carrot|              -54.0|
|  red|banana|              -32.0|
|  red| grape|              -72.0|
|  red|carrot|              -32.0|
| blue|banana|              -18.0|
+-----+------+-------------------+



In [176]:
df.groupBy(["color","fruit"]).agg((f.sum("v1") - f.avg("v2")).alias("sum_minus_avg")).show()

+-----+------+-------------+
|color| fruit|sum_minus_avg|
+-----+------+-------------+
| blue| grape|        -36.0|
|black|carrot|        -54.0|
|  red|banana|        -32.0|
|  red| grape|        -72.0|
|  red|carrot|        -32.0|
| blue|banana|        -18.0|
+-----+------+-------------+



**Sortowanie**

In [177]:
df.sort('color', 'v2').show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|black|carrot|  6| 60|
| blue|banana|  2| 20|
| blue| grape|  4| 40|
|  red|banana|  1| 10|
|  red|carrot|  3| 30|
|  red|carrot|  5| 50|
|  red|banana|  7| 70|
|  red| grape|  8| 80|
+-----+------+---+---+



In [178]:
df.sort('v1', ascending=False).show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|  red| grape|  8| 80|
|  red|banana|  7| 70|
|black|carrot|  6| 60|
|  red|carrot|  5| 50|
| blue| grape|  4| 40|
|  red|carrot|  3| 30|
| blue|banana|  2| 20|
|  red|banana|  1| 10|
+-----+------+---+---+



## Zadanie 5:

In [179]:
salaries_df.show(5)

+-----+---------+------------------+-----------------+--------------+----------------+-----------------+------------------+-----------------+------------------+--------------+----------------------+-------------------------+---------------+-------------+------------+-----------------+----------------+-------------+
|index|salary_id|     employer_name|    location_name|location_state|location_country|location_latitude|location_longitude|        job_title|job_title_category|job_title_rank|total_experience_years|employer_experience_years|annual_base_pay|signing_bonus|annual_bonus|stock_value_bonus|        comments| submitted_at|
+-----+---------+------------------+-----------------+--------------+----------------+-----------------+------------------+-----------------+------------------+--------------+----------------------+-------------------------+---------------+-------------+------------+-----------------+----------------+-------------+
|    0|        1|            opower|san francisco

Policz średnie zarobki per `location_state` i `job_title`

In [181]:
salaries_df.withColumn("annual_total", f.col("annual_base_pay") + f.col("annual_bonus")).groupby('location_state', 'job_title').avg('annual_total').show(5)

+--------------+--------------------+-----------------+
|location_state|           job_title|avg(annual_total)|
+--------------+--------------------+-----------------+
|            IL|   software engineer|         102500.0|
|          null|senior fullstack ...|            240.0|
|            CA|       web developer|         125010.0|
|            MN|senior front-end/...|             null|
|          null|senior manager, d...|         145020.0|
+--------------+--------------------+-----------------+
only showing top 5 rows



## Zadanie 6:

Policz minimalny i maksymalny `signing_bonus` dla kazdego pracodawcy

In [201]:
salaries_df.dropna(how='any', subset='signing_bonus') \
    .groupby('employer_name') \
    .agg(f.min('signing_bonus').alias('min_bonus'), f.max('signing_bonus')) \
    .show(20)

+--------------------+---------+------------------+
|       employer_name|min_bonus|max(signing_bonus)|
+--------------------+---------+------------------+
|umbc imaging rese...|      0.0|               0.0|
|               kapow|      0.0|               0.0|
|  industrial company|      0.0|               0.0|
|        definiens ag|   2000.0|            2000.0|
|                 aig|  20000.0|           20000.0|
|                xoom|      0.0|               0.0|
|             ad-tech|      0.0|               0.0|
|               elisa|      0.0|               0.0|
|norwegian tax aut...|      0.0|               0.0|
|       jamdeo canada|      0.0|               0.0|
|            turnpost|      0.0|               0.0|
|            plangrid|      0.0|               0.0|
|      cybercom group|      0.0|               0.0|
|            concepta|      0.0|               0.0|
|             philips|   6000.0|            6000.0|
|non-profit resear...|      0.0|               0.0|
| booz allen

# Joining and merging

In [206]:
df_1 = spark.createDataFrame([
    Row(a=1, b=2., c='string1'),
    Row(a=2, b=3., c='string2'),
    Row(a=4, b=5., c='string3')
])

df_2 = spark.createDataFrame([
    Row(a=1, z='last_1'),
    Row(a=2, z='last_2'),
    Row(a=6, z='last_3')
])

df_1.show()
df_2.show()

+---+---+-------+
|  a|  b|      c|
+---+---+-------+
|  1|2.0|string1|
|  2|3.0|string2|
|  4|5.0|string3|
+---+---+-------+

+---+------+
|  a|     z|
+---+------+
|  1|last_1|
|  2|last_2|
|  6|last_3|
+---+------+



In [203]:
df_1.join(df_2, on='a', how='inner').show()  # bardzo dużo możliwości łączenia!

+---+---+-------+------+
|  a|  b|      c|     z|
+---+---+-------+------+
|  1|2.0|string1|last_1|
|  2|3.0|string2|last_2|
+---+---+-------+------+



In [208]:
df_3 = spark.createDataFrame([
    Row(a=1, z='last_1'),
    Row(a=200, z='first_2'),
    Row(a=300, z='first_3')
])

df_3.show()
df_2.union(df_3).show()  # łączenie w kierunku wierszy

+---+-------+
|  a|      z|
+---+-------+
|  1| last_1|
|200|first_2|
|300|first_3|
+---+-------+

+---+-------+
|  a|      z|
+---+-------+
|  1| last_1|
|  2| last_2|
|  6| last_3|
|  1| last_1|
|200|first_2|
|300|first_3|
+---+-------+



In [209]:
df_2.subtract(df_3).show()  # odejmowanie w kierunku wierszy (odwrotność 'union')

+---+------+
|  a|     z|
+---+------+
|  6|last_3|
|  2|last_2|
+---+------+



# UDFs (User-Defined Functions)

In [215]:
from pyspark.sql.types import IntegerType  # Java jest językiem statycznie typowanym, należy podać typ zwracanego argumentu
import pyspark.sql.functions as f

def power3(value):
    return(pow(value, 3))

udfPower3 = f.udf(power3, IntegerType())  # returnType
df.select("v1", udfPower3(f.col('v1')).alias("power3")).show()

+---+------+
| v1|power3|
+---+------+
|  1|     1|
|  2|     8|
|  3|    27|
|  4|    64|
|  5|   125|
|  6|   216|
|  7|   343|
|  8|   512|
+---+------+



In [216]:
# inny sposób: DEKORATORY

import pandas
from pyspark.sql.functions import pandas_udf

@pandas_udf(IntegerType())
def pandas_plus_one(series: pd.Series) -> pd.Series:
    return series + 1

df.select(df.v1, pandas_plus_one(df.v1).name('plus')).show()

+---+----+
| v1|plus|
+---+----+
|  1|   2|
|  2|   3|
|  3|   4|
|  4|   5|
|  5|   6|
|  6|   7|
|  7|   8|
|  8|   9|
+---+----+



# Spark SQL

In [220]:
df.show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|  red|banana|  1| 10|
| blue|banana|  2| 20|
|  red|carrot|  3| 30|
| blue| grape|  4| 40|
|  red|carrot|  5| 50|
|black|carrot|  6| 60|
|  red|banana|  7| 70|
|  red| grape|  8| 80|
+-----+------+---+---+



In [218]:
df.createOrReplaceTempView("my_table")  # UWAGA: wymagany jest zarejestrowany widok tabeli
spark.sql("SELECT count(*) from my_table").show()  # składnia SQL

+--------+
|count(1)|
+--------+
|       8|
+--------+



In [221]:
@pandas_udf("integer")  # alias dla 'IntegerType()''
def add_one(s: pd.Series) -> pd.Series:
    return s + 1

spark.udf.register("add_one", add_one)
spark.sql("SELECT add_one(v1) FROM my_table").show()

+-----------+
|add_one(v1)|
+-----------+
|          2|
|          3|
|          4|
|          5|
|          6|
|          7|
|          8|
|          9|
+-----------+



In [222]:
from pyspark.sql.functions import expr

df.selectExpr('add_one(v1)').show()
df.select(expr('count(*)') > 0).show()
df.selectExpr("v1 * 2", "abs(v2)").show()

+-----------+
|add_one(v1)|
+-----------+
|          2|
|          3|
|          4|
|          5|
|          6|
|          7|
|          8|
|          9|
+-----------+

+--------------+
|(count(1) > 0)|
+--------------+
|          true|
+--------------+

+--------+-------+
|(v1 * 2)|abs(v2)|
+--------+-------+
|       2|     10|
|       4|     20|
|       6|     30|
|       8|     40|
|      10|     50|
|      12|     60|
|      14|     70|
|      16|     80|
+--------+-------+



# Ćwiczenia

In [210]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [267]:
spark = SparkSession.builder.appName('my_app').master("local[*]").getOrCreate()

goBike = spark.read.csv("./2017-fordgobike-tripdata.csv", header=True, inferSchema=True)
goBike.printSchema()

root
 |-- duration_sec: integer (nullable = true)
 |-- start_time: timestamp (nullable = true)
 |-- end_time: timestamp (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: double (nullable = true)
 |-- start_station_longitude: double (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: double (nullable = true)
 |-- end_station_longitude: double (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: integer (nullable = true)
 |-- member_gender: string (nullable = true)



In [231]:
goBike.show(3, vertical=True)

-RECORD 0---------------------------------------
 duration_sec            | 80110                
 start_time              | 2017-12-31 16:57:... 
 end_time                | 2018-01-01 15:12:... 
 start_station_id        | 74                   
 start_station_name      | Laguna St at Haye... 
 start_station_latitude  | 37.776434819204745   
 start_station_longitude | -122.42624402046204  
 end_station_id          | 43                   
 end_station_name        | San Francisco Pub... 
 end_station_latitude    | 37.7787677           
 end_station_longitude   | -122.4159292         
 bike_id                 | 96                   
 user_type               | Customer             
 member_birth_year       | 1987                 
 member_gender           | Male                 
-RECORD 1---------------------------------------
 duration_sec            | 78800                
 start_time              | 2017-12-31 15:56:... 
 end_time                | 2018-01-01 13:49:... 
 start_station_id   

**Zapoznaj się z danymi:**

> 1. Pozbądź się wierszy zawierających NULLe

In [232]:
for column in goBike.columns:
    
    nan_count = goBike.where(f.col(column).isNull()).count()
    print(f'{column}: {nan_count}')

duration_sec: 0


start_time: 0


end_time: 0


start_station_id: 0


start_station_name: 0


...

end_station_longitude: 0


bike_id: 0


user_type: 0


member_birth_year: 66541


member_gender: 66462


In [268]:
goBike = goBike.dropna(how='any')

# for column in goBike.columns:
    
#     nan_count = goBike.where(f.col(column).isNull()).count()
#     print(f'{column}: {nan_count}')

> 2. Sprawdź rozkład zmiennej "member_gender"

In [269]:
goBike.printSchema()

goBike.groupby('member_gender').count().show()

root
 |-- duration_sec: integer (nullable = true)
 |-- start_time: timestamp (nullable = true)
 |-- end_time: timestamp (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: double (nullable = true)
 |-- start_station_longitude: double (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: double (nullable = true)
 |-- end_station_longitude: double (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: integer (nullable = true)
 |-- member_gender: string (nullable = true)



+-------------+------+
|member_gender| count|
+-------------+------+
|       Female| 98542|
|        Other|  6299|
|         Male|348318|
+-------------+------+



> 3. Oblicz minimalny, maksymalny i średni wiek osób wypożyczających rowery

In [270]:
goBike = goBike.withColumn('age', 2023 - f.col('member_birth_year'))
goBike.show(5)

goBike.agg(f.min('age'), f.max('age'), f.avg('age')).show(5)

+------------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+---+
|duration_sec|          start_time|            end_time|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id| user_type|member_birth_year|member_gender|age|
+------------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+---+
|       80110|2017-12-31 16:57:...|2018-01-01 15:12:...|              74|Laguna St at Haye...|    37.776434819204745|    -122.42624402046204|            43|San Francisco Pu

In [272]:
# inny sposób:
goBike.select((2023 - goBike.member_birth_year).alias('age')).agg(f.min('age'), f.max('age'), f.avg('age')).show()

+--------+--------+------------------+
|min(age)|max(age)|          avg(age)|
+--------+--------+------------------+
|      24|     137|42.595212717831934|
+--------+--------+------------------+



> 4. Oblicz liczbę unikalnych rowerów

In [308]:
# goBike.select('bike_id').count()
goBike.select('bike_id').distinct().count()

3670

> 5. Oblicz liczbę unikalnych stacji

In [285]:
goBike.select(goBike.start_station_id) \
    .union(goBike.select(goBike.end_station_id)) \
    .distinct() \
    .count()

272

> 6. Sprawdź który rower był wypożyczony najdłużej, a który najkrócej w ciągu analizowanego okresu (oraz jak długo)

In [310]:
goBike.groupby('bike_id') \
    .agg(f.sum("duration_sec").alias("total_duration")) \
    .sort("total_duration", ascending=False) \
    .show(1)

+-------+--------------+
|bike_id|total_duration|
+-------+--------------+
|   2178|        377954|
+-------+--------------+
only showing top 1 row



In [312]:
goBike.groupby('bike_id') \
    .agg(f.sum("duration_sec").alias("total_duration")) \
    .orderBy("total_duration") \  # 'orderBy' działa tu analogicznie do 'sort'
    .show(1)  

+-------+--------------+
|bike_id|total_duration|
+-------+--------------+
|   2609|            74|
+-------+--------------+
only showing top 1 row



> 7. Oblicz średni czas pojedynczego wypożyczenia

In [331]:
# goBike.agg(f.avg('duration_sec')).show()
goBike.select(f.round(f.avg('duration_sec'))).show()  # zaokrąglenie do liczby całkowitej

+---------------------------+
|round(avg(duration_sec), 0)|
+---------------------------+
|                      833.0|
+---------------------------+



> 8. Sprawdź pomiędzy którymi stacjami występował największy ruch (__hint__: `A -> B == B -> A`)

In [332]:
goBike.withColumn("route", f.when(f.col('start_station_id') < f.col('end_station_id'), f.array([f.col('start_station_id'), f.col('end_station_id')])) \
            .otherwise(f.array([f.col('end_station_id'), f.col('start_station_id')]))) \
            .select("start_station_id", "end_station_id", "route").show()

+----------------+--------------+----------+
|start_station_id|end_station_id|     route|
+----------------+--------------+----------+
|              74|            43|  [43, 74]|
|             284|            96| [96, 284]|
|             239|           247|[239, 247]|
|             259|           259|[259, 259]|
|              67|            24|  [24, 67]|
|              67|            24|  [24, 67]|
|              14|            27|  [14, 27]|
|              14|            27|  [14, 27]|
|              78|            15|  [15, 78]|
|               4|           123|  [4, 123]|
|             317|           296|[296, 317]|
|             139|            99| [99, 139]|
|             323|            14| [14, 323]|
|             323|            14| [14, 323]|
|             251|           259|[251, 259]|
|               7|           230|  [7, 230]|
|              96|            86|  [86, 96]|
|             278|           314|[278, 314]|
|             182|           195|[182, 195]|
|         

In [333]:
goBike.withColumn("route", f.when(f.col('start_station_id') < f.col('end_station_id'), f.array([f.col('start_station_id'), f.col('end_station_id')])) \
            .otherwise(f.array([f.col('end_station_id'), f.col('start_station_id')]))) \
            .groupby("route") \
            .count() \
            .orderBy('count', ascending=False) \
            .show(10)

+----------+-----+
|     route|count|
+----------+-----+
|   [6, 15]| 3282|
|  [15, 81]| 2953|
|  [21, 48]| 2939|
|   [6, 16]| 2840|
|  [15, 50]| 2220|
|[182, 196]| 2153|
|[182, 195]| 1910|
|  [25, 30]| 1811|
|  [22, 30]| 1746|
|  [58, 67]| 1745|
+----------+-----+
only showing top 10 rows



> 9. Sprawdź, o której godzinie w ciągu dnia wypożyczano najwięcej rowerów

In [335]:
goBike.select(f.hour(goBike.start_time).alias('start_hour')) \  # f.hour() wydobywa godzinę z obiektu typu 'timestamp'
    .groupby('start_hour') \
    .count() \
    .orderBy('count', ascending=False) \
    .show()

+----------+-----+
|start_hour|count|
+----------+-----+
|         8|54678|
|        17|54620|
|         9|42184|
|        18|41752|
|        16|35478|
|         7|25656|
|        19|23552|
|        15|22491|
|        12|21760|
|        13|21155|
|        10|20999|
|        11|19139|
|        14|18902|
|        20|14628|
|        21|10362|
|         6| 8520|
|        22| 6689|
|        23| 4012|
|         0| 2105|
|         5| 2018|
+----------+-----+
only showing top 20 rows



> 10. Sprawdź *średnią liczbę wypożyczeń* dla poszczególnych dni tygodnia (__hint__: `java.text.SimpleDateFormat`)

In [338]:
goBike.select(f.date_format("start_time", "E").alias("weekday"),
             f.date_format("start_time", "dd.MM.yyyy").alias("date")).show()

+-------+----------+
|weekday|      date|
+-------+----------+
|    Sun|31.12.2017|
|    Sun|31.12.2017|
|    Sun|31.12.2017|
|    Sun|31.12.2017|
|    Sun|31.12.2017|
|    Sun|31.12.2017|
|    Sun|31.12.2017|
|    Sun|31.12.2017|
|    Sun|31.12.2017|
|    Sun|31.12.2017|
|    Sun|31.12.2017|
|    Sun|31.12.2017|
|    Sun|31.12.2017|
|    Sun|31.12.2017|
|    Sun|31.12.2017|
|    Sun|31.12.2017|
|    Sun|31.12.2017|
|    Sun|31.12.2017|
|    Sun|31.12.2017|
|    Sun|31.12.2017|
+-------+----------+
only showing top 20 rows



In [358]:
# goBike.withColumn("weekday", f.date_format("start_time", "E")).groupby("weekday").count().show()

goBike.select(f.date_format("start_time", "E").alias("weekday"), f.date_format("start_time", "dd.MM.yyyy").alias("date")) \
    .groupby("weekday", "date") \
    .count() \
    .groupby("weekday") \
    .agg(f.round(f.avg("count")).alias("avg_bikes")) \
    .show()

+-------+---------+
|weekday|avg_bikes|
+-------+---------+
|    Sun|   1214.0|
|    Mon|   2828.0|
|    Thu|   2859.0|
|    Sat|   1362.0|
|    Wed|   2976.0|
|    Tue|   3095.0|
|    Fri|   2667.0|
+-------+---------+

